In [1]:
from google.colab import drive
import os
import re
from collections import Counter

# Google Drive verbinden
drive.mount('/content/drive')


# Liste der Risiko-Wörter
risiko = ["risik", "wahrscheinlich", "poten", "eintritt", "eingetreten", "ausmaß", "schad", "schäd", "erwart", "verletz", "schutz", "vulnerab",
          "katastroph", "bewältig", "anpass", "angepasst", "tot", "gestorben", "sterb", "ausgesetzt", "aussetz", "verlust", "verlier",
          "verlor", "fehler", "extrem", "knapp", "bedroh", "schutz", "sozi", "technologi", "ökonom", "gesundheit",
          "maßnahme", "schuld", "fehler", "zukunft", "zukünft", "vorhersag", "einschätz", "unsicher", "gefahr", "gefähr", "individ", "privat",
          "politi", "bund", "region", "kommun", "lokal", "versicher", "prävent", "verantwor"]

# Funktion, um die relative Häufigkeit der Risiko-Wörter in einem Dokument zu berechnen
def count_risk_words_in_doc(doc_path, risk_words):
    try:
        with open(doc_path, 'r', encoding='utf-8') as file:
            content = file.read().lower()  # Lese das Dokument und mache den Text klein (case insensitive)
            words = re.findall(r'\b\w+\b', content)  # Alle Wörter im Dokument extrahieren
            total_words = len(words)  # Gesamtanzahl der Wörter im Dokument

            # Zähle nur die Risiko-Wörter
            word_counts = Counter([word for word in words if any(risk_word in word for risk_word in risk_words)])
            risk_word_count = sum(word_counts.values())  # Anzahl der Risiko-Wörter

            # Berechne die relative Häufigkeit der Risiko-Wörter (Anteil)
            if total_words > 0:
                relative_risk = risk_word_count / total_words
            else:
                relative_risk = 0  # Falls das Dokument keine Wörter enthält

            return relative_risk  # Gib den relativen Anteil zurück

    except Exception as e:
        print(f"Fehler beim Lesen der Datei {doc_path}: {e}")
        return 0

# Funktion, um alle .txt-Dateien in einem Verzeichnis und dessen Unterordnern zu durchsuchen
def scan_directory_for_risk_words(directory, risk_words, min_chars=1000):
    file_risk_counts = []

    # Durchsuche alle Unterordner und Dateien im angegebenen Verzeichnis
    for subdir, _, files in os.walk(directory):
        for file in files:
            if file.endswith('.txt'):
                file_path = os.path.join(subdir, file)

                # Überprüfe, ob die Datei mehr als min_chars Zeichen hat
                with open(file_path, 'r', encoding='utf-8') as f:
                    content = f.read()
                    if len(content) >= min_chars:
                        # Berechne die relative Häufigkeit der Risiko-Wörter in der Datei
                        relative_risk = count_risk_words_in_doc(file_path, risk_words)
                        file_risk_counts.append((file_path, relative_risk))

    return file_risk_counts

# Funktion, um eine Rangliste der Dokumente nach relativer Risiko-Wort-Häufigkeit zu erstellen
def create_ranking_and_save(file_risk_counts, output_path):
    # Sortiere die Dateien nach der relativen Häufigkeit der Risiko-Wörter (absteigend)
    sorted_file_risk_counts = sorted(file_risk_counts, key=lambda x: x[1], reverse=True)

    # Speichere die Rangliste in einer neuen Textdatei
    with open(output_path, 'w', encoding='utf-8') as output_file:
        for file_path, risk_percentage in sorted_file_risk_counts:
            output_file.write(f"{file_path}: {risk_percentage:.4f} (relative Häufigkeit)\n")

    print(f"Die Rangliste wurde in {output_path} gespeichert.")

# Beispiel: Verzeichnis und Ausgabe-Dateipfad
input_directory = '/content/drive/MyDrive/Zeitungsartikel_final'  # Ersetze dies durch den Pfad zu deinem Zielordner
output_file_path = '/content/drive/MyDrive/Zeitungsartikel_final/risk_ranking1000.txt'  # Ersetze dies durch den gewünschten Ausgabepfad

# Hauptteil des Codes
file_risk_counts = scan_directory_for_risk_words(input_directory, risiko)
create_ranking_and_save(file_risk_counts, output_file_path)

# Beispiel: Verzeichnis und Ausgabe-Dateipfad
input_directory = '/content/drive/MyDrive/Zeitungsartikel_final'  # Ersetze dies durch den Pfad zu deinem Zielordner
output_file_path = '/content/drive/MyDrive/Zeitungsartikel_final/risk_ranking.txt'  # Ersetze dies durch den gewünschten Ausgabepfad


MessageError: Error: credential propagation was unsuccessful